In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split

from pathlib import Path

import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

In [ ]:
data_dir = Path("../Test-Task-for-DS-time-series-forecasting-2026-01/data")
sales = data_dir / "ts_hist.csv"

print("sales:", sales.exists())

In [ ]:
df = pd.read_csv(sales)
print(df.shape, "\n")
df.head()

In [ ]:
df.info()

In [ ]:
for name in df.columns:
    n_unique = df[name].nunique()
    n_missing = df[name].isna().sum()
    dtype = df[name].dtype
    print(f"{name:15} | {n_unique:10} | {n_missing:10} | {dtype}")

In [ ]:
columsns = ['cat_id', 'store_id', 'dept_id', 'event_name_1', 'event_name_2']
for val in columsns:
    print(f"{val}: {df[val].unique()}")

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df['date'].min(), df['date'].max()

In [ ]:
df = df.sort_values(by=['index', 'date']).reset_index(drop=True)
df.head()

In [ ]:
df.duplicated(subset=['index', 'date']).sum()

In [ ]:
df_kernel = df[['index', 'date', 'qnt']]
df_kernel.rename(columns={'qnt': 'y'}, inplace=True)
df_kernel.info()

In [ ]:
df_kernel.head()

In [ ]:
# Check time differences
gap_counts = df_kernel.groupby('index')['date'].diff().value_counts()
gap_counts.sort_index()

In [ ]:
df_kernel.groupby('date')['y'].sum().reset_index(drop=True)
df_kernel = df_kernel.set_index('index')
df_kernel

In [ ]:
# Візуалізація пропусків у даних за допомогою теплової карти
unique_items = df_kernel['index'].unique()[51:100] 
df_sample = df_kernel[df_kernel['index'].isin(unique_items)].copy()

# 2. Створення матриці (Pivot Table)
# Рядки = Товари, Стовпці = Дати, Значення = Є дані чи ні
# pivot автоматично створить NaN (порожнечу) там, де не було рядків у датасеті
matrix = df_sample.pivot(index='index', columns='date', values='y')

# 3. Перетворення на булеву матрицю (1 = Є дані, 0 = Немає)
# Це потрібно, щоб колір залежав тільки від наявності запису, а не від кількості продажів
sparsity_matrix = matrix.notna().astype(int)

# 4. Візуалізація
plt.figure(figsize=(15, 10))

# Використовуємо cmap='binary', де 0 (немає даних) буде білим, а 1 (є дані) — чорним
sns.heatmap(sparsity_matrix, 
            cbar=False,           # Прибираємо шкалу кольорів, вона тут не треба
            cmap='binary',        # Чорно-біла палітра
            xticklabels=50,       # Показувати дату тільки кожні 50 днів, щоб не забивати вісь
            yticklabels=True)     # Показувати назви товарів

plt.title('Карта наявності історичних даних (Чорне = є запис, Біле = пропуск)', fontsize=14)
plt.xlabel('Дата')
plt.ylabel('Товари (index)')
plt.show()